In [1]:
import numpy as np
import os
import pandas as pd
import pickle
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization

In [2]:
DATASET_DIR = "dataset"

image_paths = []
labels = []

for label in os.listdir(DATASET_DIR):
    label_dir = os.path.join(DATASET_DIR, label)
    if not os.path.isdir(label_dir):
        continue

    for img_file in os.listdir(label_dir):
        image_paths.append(os.path.join(label_dir, img_file))
        labels.append(label)

df = pd.DataFrame({
    "image": image_paths,
    "label": labels
})

num_classes = df["label"].nunique()

train, test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

# TEMPORARY: limit dataset size to avoid crashing the machine
train = train.sample(n=min(15000, len(train)), random_state=42)
test = test.sample(n=min(3000, len(test)), random_state=42)

In [3]:
# funkcija prolazi kroz direktorij sa slikama
# svaka slika se nalazi u podfolderu cije ime predstavlja labelu
# sprema pune putanje do svih slika u listu image_paths
# i pripadajuce labele u listu labels
# na kraju vraca te dvije liste
def extract_features(images):
    features= []
    for image in tqdm(images):
        img = load_img(image, color_mode="grayscale", target_size=(48, 48))
        img=np.array(img)
        features.append(img)
    features=np.array(features)
    features=features.reshape(len(features), 48, 48, 1)
    return features

In [4]:
train_features = extract_features(train['image'])

  0%|          | 0/15000 [00:00<?, ?it/s]

In [5]:
test_features = extract_features(test['image'])

  0%|          | 0/3000 [00:00<?, ?it/s]

In [6]:
x_train=train_features/255.0 
x_test=test_features/255.0 

In [7]:
le = LabelEncoder()
le.fit(train['label'])

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

In [8]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [9]:
y_train = to_categorical(y_train, num_classes = num_classes)
y_test = to_categorical(y_test, num_classes = num_classes)

In [10]:
input_shape = (48, 48, 1)

model = Sequential([
    Input(shape=input_shape)
])

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

#fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(x=x_train, y=y_train, batch_size=64, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 106s 440ms/step - accuracy: 0.2666 - loss: 1.8962 - val_accuracy: 0.3030 - val_loss: 1.5944
Epoch 2/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 159s 678ms/step - accuracy: 0.3330 - loss: 1.5202 - val_accuracy: 0.3423 - val_loss: 1.5630
Epoch 3/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 192s 634ms/step - accuracy: 0.4235 - loss: 1.3599 - val_accuracy: 0.3650 - val_loss: 1.5966
Epoch 4/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 216s 693ms/step - accuracy: 0.4698 - loss: 1.2669 - val_accuracy: 0.3810 - val_loss: 1.5334
Epoch 5/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 162s 689ms/step - accuracy: 0.5075 - loss: 1.1828 - val_accuracy: 0.4377 - val_loss: 1.3103
Epoch 6/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 164s 698ms/step - accuracy: 0.5409 - loss: 1.1190 - val_accuracy: 0.4360 - val_loss: 1.3940
Epoch 7/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 162s 691ms/step - accuracy: 0.5683 - loss: 1.0629 - val_accuracy: 0.5277 - val_loss: 1.1873
Epoch 8/15
235/235 ━━━━━━━━━━━━━━━━━━━━ 169s 718ms/step - accuracy: 0.5838 -

In [13]:
model.save("emotion_cnn_48x48.keras")